# Importando dados da api _Seeking Alpha_

In [30]:
import pandas as pd
import requests
import json
import time


from api_key import rapidapi

### Recebendo e tabulando os dados da API

##### def recebeDados()
    A função _recebeDados_ recebe um parâmetro "registroBVMF". A função faz uma solicitação GET para a API "seeking-alpha.p.rapidapi.com" usando a biblioteca "requests", e armazena a resposta JSON em "response". Em seguida, a função extrai dados específicos da resposta JSON e armazena-os em variáveis separadas.
    A função retorna uma tupla contendo essas variáveis

In [31]:
def recebeDados(registroBVMF):
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-profile"
    querystring = {"symbols": registroBVMF}

    headers = {
        "X-RapidAPI-Key": rapidapi,
        "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring).json()
    
    dados = response['data'][0]['attributes']
    descricao = dados['companyName']
    last_daily = dados['lastDaily']
    cotacao_open = last_daily['open']
    cotacao_close = last_daily['close']
    cotacao_low = last_daily['low']
    cotacao_high = last_daily['high']
    
    return descricao, cotacao_open, cotacao_close, cotacao_low, cotacao_high


##### def criaDF()
    A função recebe uma lista de empresas, itera sobre elas obtendo seus dados e cria um dataframe com esses dados, retornando-o. A função utiliza um dicionário para armazenar os dados e adiciona um atraso de 0.2 segundos entre as solicitações pela limitação de dados por segundo da API.

In [45]:
def get_dados_empresas(lista_empresas):
    # Dicionário para os dados das empresas
    dados_empresas = {}

    # Itera sobre a lista de empresas
    for empresa in lista_empresas:
        descricao, cotacao_open, cotacao_close, cotacao_low, cotacao_high = recebeDados(empresa)
        dados_empresas[empresa] = {'e_id': empresa, 'descricao': descricao, 'cotacao_open': cotacao_open, 'cotacao_close': cotacao_close, 'cotacao_low': cotacao_low, 'cotacao_high': cotacao_high}
        time.sleep(0.2) # Adiciona um atraso de 0.2 segundos (5 solicitações por segundo)

    # Cria dataframe
    df = pd.DataFrame.from_dict(dados_empresas, orient='index', columns=['e_id', 'descricao', 'cotacao_open', 'cotacao_close', 'cotacao_low', 'cotacao_high'])
    df = df.reset_index()
    df = df.drop('e_id', axis=1)

    return df

In [50]:
lista_empresas = ["AAPL", "AMZN", "FB", "GOOGL", "MSFT", "NFLX", "TSLA", "JPM", "V", "WMT"]
df_cotacao = get_dados_empresas(lista_empresas)
display(df_cotacao)

,index,descricao,cotacao_open,cotacao_close,cotacao_low,cotacao_high
0,AAPL,Apple Inc.,161.630,165.56,161.42,165.80
1,AMZN,"Amazon.com, Inc.",98.950,102.40,98.71,102.57
2,FB,"Meta Platforms, Inc.",194.550,196.64,194.41,202.03
3,GOOGL,Alphabet Inc.,105.840,107.43,105.84,107.49
4,MSFT,Microsoft Corporation,283.590,289.84,283.17,289.90
5,NFLX,"Netflix, Inc.",339.990,346.19,338.75,346.43
6,TSLA,"Tesla, Inc.",182.955,185.90,180.94,186.50
7,JPM,JPMorgan Chase & Co.,128.460,128.99,126.83,129.04
8,V,Visa Inc.,229.460,232.69,228.80,232.84
9,WMT,Walmart Inc.,149.650,149.49,148.20,150.00


In [51]:
df_cotacao.to_csv('cotacao.csv')